In [1]:
from PIL import Image
from glob import glob
import os

In [2]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [3]:
DATA_DIR = './examples/'
OUT_DIR = './test_images'
DATA_TMPL = '*'
MIN_BLOCK_SIZE = 5

In [4]:
class codec(object):
    
    def __init__(self, codec, params):
        self.codec = codec
        self.params = params

In [5]:
out_fromats = [
    {'format': 'jpeg', 'params': {'quality': 20}},
    {'format': 'jpeg', 'params': {'quality': 40}},
    {'format': 'jpeg', 'params': {'quality': 60}},
    {'format': 'jpeg', 'params': {'quality': 80}},
    {'format': 'jpeg', 'params': {'optimize': True}},
    {'format':  'png', 'params': {'compress_level': 2}},
    {'format':  'png', 'params': {'compress_level': 4}},
    {'format':  'png', 'params': {'compress_level': 6}},
    {'format':  'png', 'params': {'compress_level': 8}},
    {'format':  'png', 'params': {'optimize': True}},
    {'format': 'webp', 'params': {'quality': 20}},
    {'format': 'webp', 'params': {'quality': 40}},
    {'format': 'webp', 'params': {'quality': 60}},
    {'format': 'webp', 'params': {'quality': 80}},
    {'format': 'webp', 'params': {'lossless': True}}
]

In [6]:
img_files = sorted(glob(os.path.join(DATA_DIR, DATA_TMPL)))

In [ ]:
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

for img_file in log_progress(img_files[:]):
    img = Image.open(img_file)
    print '{}, {}, {}'.format(img.format, img.mode, img.size)
    height_mp = int(round(img.height / 32.0))
    width_mp = int(round(img.width / 32.0))
    w_add = h_add = 0
    if width_mp > height_mp:
        w_add = width_mp - height_mp
    else:
        h_add = height_mp - width_mp
    for mp in range(MIN_BLOCK_SIZE, min(height_mp, width_mp) + 1):
        img_res = img.resize((32 * (mp + w_add), 32 * (mp + h_add)))
        for out_frm in out_fromats:
            out_name = '{}_size={}x{},codec={}'.format(os.path.basename(img_file).rsplit('.')[0],
                                                       img_res.width, img_res.height, out_frm['format'])
            for p, v in out_frm['params'].iteritems():
                out_name += ',{}={}'.format(p, v)
            img_res.save(os.path.join(OUT_DIR, out_name), format=out_frm['format'], **out_frm['params'])
        out_name = '{}_size={}x{}_orig'.format(os.path.basename(img_file).rsplit('.')[0], img_res.width,
                                               img_res.height)
        img_res.save(os.path.join(OUT_DIR, out_name), format='png', compress_level=0)
#         break
#     break

PNG, RGB, (490, 733)
PNG, RGB, (1118, 1105)
PNG, RGB, (768, 512)
PNG, RGB, (512, 512)
PNG, RGB, (1024, 768)
